# Practice Python JNB

Let's start by importing in the `R` function. We use `rpy2` to import the function and use it in `Python`.

In [1]:
import rpy2.robjects as ro
import pandas as pd
import json

r = ro.r
r['source']('ParseNFLPlayers.R')

parse_nfl_player_r = ro.globalenv['get_player_data']
parse_position_players_r = ro.globalenv['get_position_players']

# now we can use the function parse_nfl_player_r() to get info on players
# parse_nfl_player_r() returns an array of length one with the JSON data
# use parse_nfl_player_r("player_name")[0] to get a string of the JSON

# parse_position_player_r() to get info on positions
# use parse_position_players_r("position")[0] to get a string of the JSON

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


R[write to console]: ── Attaching packages ──────────────────────────────────────── nflverse 1.0.1 ──

R[write to console]: ✔ nflfastR 4.3.0     ✔ nflreadr 1.1.3
✔ nflseedR 1.0.2     ✔ nflplotR 1.0.0
✔ nfl4th   1.0.1     

R[write to console]: ──────────────────────────────────────────────────────────────── Ready to go! ──

R[write to console]: Registered S3 method overwritten by 'tune':
  method                   from   
  required_pkgs.model_spec parsnip

R[write to console]: ── Attaching packages ────────────────────────────────────── tidymodels 0.1.4 ──

R[write to console]: ✔ broom        0.7.12     ✔ recipes      0.1.17
✔ dials        0.0.10     ✔ rsample      0.1.1 
✔ dplyr        1.0.7      ✔ tibble       3.1.6 
✔ ggplot2      3.3.5      ✔ tidyr        1.1.4 
✔ infer        1.0.0      ✔ tune         0.1.6 
✔ modeldata    0.1.1      ✔ workflows    0.2.4 
✔ parsnip      0.1.7      ✔ workflowsets 0.1.0 
✔ purrr        0.3.4      ✔ yardstick    0.0.9 

R[write to console]: ── Confl

Here is an example of manipulating some player data with `pandas`. Let's look at the data for Patrick Mahomes.

In [2]:
# Get the data from the R function
string_json_pmahomes = parse_nfl_player_r("Patrick Mahomes")[0]
# Parse the string into a JSON
json_pmahomes = json.loads(string_json_pmahomes)
print(json_pmahomes)

{'team': ['KC'], 'position': ['QB'], 'depth_chart_position': ['QB'], 'jersey_number': [15], 'status': ['Active'], 'full_name': ['Patrick Mahomes'], 'first_name': ['Patrick'], 'last_name': ['Mahomes'], 'birth_date': ['1995-09-17'], 'height': ['75'], 'weight': ['227'], 'college': ['Texas Tech'], 'high_school': ['Whitehouse (TX)'], 'gsis_id': ['00-0033873'], 'espn_id': [3139477], 'sportradar_id': ['11cad59d-90dd-449c-a839-dddaba4fe16c'], 'yahoo_id': [30123], 'rotowire_id': [11839], 'pff_id': [11765], 'pfr_id': ['MahoPa00'], 'fantasy_data_id': [18890], 'sleeper_id': ['4046'], 'years_exp': [5], 'headshot_url': ['https://static.www.nfl.com/image/private/f_auto,q_auto/league/tlg2oo5yrk6mins3nxxe'], 'week': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21], 'completions': [27, 24, 27, 24, 33, 32, 20, 29, 20, 35, 23, 15, 20, 31, 23, 26, 27, 30, 33, 26], 'attempts': [36, 31, 44, 30, 54, 47, 35, 48, 37, 50, 37, 29, 24, 47, 30, 35, 44, 39, 44, 39], 'passing_yards': [337, 343,

In [10]:
# Turn the JSON into a dataframe
df = pd.json_normalize(json_pmahomes)
# Display the dataframe
df.head()

,team,position,depth_chart_position,jersey_number,status,full_name,first_name,last_name,birth_date,height,...,team_division,team_color,team_color2,team_color3,team_color4,team_logo_wikipedia,team_logo_espn,team_wordmark,team_conference_logo,team_league_logo
0,[KC],[QB],[QB],[15],[Active],[Patrick Mahomes],[Patrick],[Mahomes],[1995-09-17],[75],...,[AFC West],[#E31837],[#FFB612],[#000000],[#e31837],[https://upload.wikimedia.org/wikipedia/en/thu...,[https://a.espncdn.com/i/teamlogos/nfl/500/kc....,[https://github.com/nflverse/nflfastR-data/raw...,[https://github.com/nflverse/nflfastR-data/raw...,[https://raw.githubusercontent.com/nflverse/nf...


(The dataframe is pretty small - I hope to change the API soon to return more data rather than just averages.)

Instead, let's look at the data directly in `Python`. We get this from the same `R` file that was imported in the first code chunk.

In [2]:
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# This is an R "tibble" of the data
# We load this object from the R script we used earlier
player_stats_r = ro.globalenv['player_stats']
team_data_r = ro.globalenv['team_data']
player_stats_kicker_r = ro.globalenv['player_stats_kicker']
roster_r = ro.globalenv['roster']
official_player_stats_r = ro.globalenv['official_player_stats']
nextgen_stats_r = ro.globalenv['nextgen_stats']
combine_r = ro.globalenv['combine']

# Convert the tibble to a pandas data frame
with localconverter(ro.default_converter + pandas2ri.converter):
    df_player_stats = ro.conversion.rpy2py(player_stats_r)
    df_team_data = ro.conversion.rpy2py(player_stats_r)
    df_player_stats_kicker = ro.conversion.rpy2py(player_stats_kicker_r)
    df_roster = ro.conversion.rpy2py(roster_r)
    df_official_player_stats = ro.conversion.rpy2py(official_player_stats_r)
    df_nextgen_stats = ro.conversion.rpy2py(nextgen_stats_r)
    df_combine = ro.conversion.rpy2py(combine_r)

# View the first 5 rows of the data
df_official_player_stats.head(100)

,player_id,player_name,games,recent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,...,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
1,00-0019596,T.Brady,17,TB,485,719,5316.0,43,12.0,22.0,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,374.74,374.74
2,00-0022824,A.Lee,1,ARI,0,0,0.0,0,0.0,0.0,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,-2.00,-2.00
3,00-0022924,B.Roethlisberger,16,PIT,390,605,3740.0,22,10.0,38.0,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,218.10,218.10
4,00-0023459,Aa.Rodgers,16,GB,366,531,4115.0,37,4.0,30.0,...,0.0,-1.243446,0,0.000000,0.032258,-0.011299,0.040477,0.0,332.30,333.30
5,00-0023682,R.Fitzpatrick,1,WAS,3,6,13.0,0,0.0,1.0,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.72,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,00-0031363,M.Palardy,1,MIA,0,0,0.0,0,0.0,0.0,...,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,0.10,0.10
97,00-0031376,J.McKinnon,7,KC,0,0,0.0,0,0.0,0.0,...,7.0,2.175842,0,-11.888889,0.065841,-0.006947,0.093898,0.0,22.90,35.90
98,00-0031381,D.Adams,16,GB,0,0,0.0,0,0.0,0.0,...,84.0,71.113759,0,0.969413,0.316801,0.370177,0.734326,0.0,221.30,344.30
99,00-0031382,J.Landry,12,CLE,0,0,0.0,0,0.0,1.0,...,28.0,-13.182096,0,0.841950,0.244817,0.246103,0.539497,0.0,81.00,133.00


Have fun :) If you want to check out other dataframes, `player_stats_kicking`, `team_data`, and `roster` are all tibbles in the `R` script. Import them into python using the same process that we used for `player_stats`.

Let me know if you want me to add some examples of how to manipulate data using Python!

In [25]:
df_player_stats[df_player_stats.recent_team == "TB"]
df_player_stats[["recent_team", "week"]]

,recent_team,week
1,TB,1
2,TB,2
3,TB,3
4,TB,4
5,TB,5
...,...,...
5676,NYJ,14
5677,NYJ,15
5678,NYJ,16
5679,NYJ,17
